# Erstellen einer FHIR Observation Ressource mit dem Python fhirclient zur Angabe der Herzfrequenz
Um den Python fhirclient zu installieren kann man entweder die aktuellste Version aus dem [GitHub Repository](https://github.com/smart-on-fhir/client-py) nehmen oder die zuletzt auf [PyPi](https://pypi.org/project/fhirclient/) veröffentlichte Version. Da diese jedoch noch STU-3 ist, nutzen wir für die folgenden Beispiele die GitHub Version (R4).

In [ ]:
!pip install git+https://github.com/smart-on-fhir/client-py.git

In [1]:
from IPython.display import IFrame
import json
from fhirclient.models import (
    patient,
    observation
)

In [2]:
# Instanziieren einer Observation
my_observation = observation.Observation()

## Mit der Observation Ressource vertraut machen
Um erstmal einen Überblick davon zu erhalten wie die Observation Ressource aufgebaut ist, bzw. welche Attribute diese enthält kann die Funktion **elementProperties()** aufgerufen werden, diese gibt dann alle Attribute der fhirclient Klasse **Observation()** mit ihren jeweiligen Eigenschaften zurück.

In [3]:
for property in my_observation.elementProperties():
    print(property)

('id', 'id', <class 'str'>, False, None, False)
('implicitRules', 'implicitRules', <class 'str'>, False, None, False)
('language', 'language', <class 'str'>, False, None, False)
('meta', 'meta', <class 'fhirclient.models.meta.Meta'>, False, None, False)
('contained', 'contained', <class 'fhirclient.models.resource.Resource'>, True, None, False)
('extension', 'extension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('modifierExtension', 'modifierExtension', <class 'fhirclient.models.extension.Extension'>, True, None, False)
('text', 'text', <class 'fhirclient.models.narrative.Narrative'>, False, None, False)
('basedOn', 'basedOn', <class 'fhirclient.models.fhirreference.FHIRReference'>, True, None, False)
('bodySite', 'bodySite', <class 'fhirclient.models.codeableconcept.CodeableConcept'>, False, None, False)
('category', 'category', <class 'fhirclient.models.codeableconcept.CodeableConcept'>, True, None, False)
('code', 'code', <class 'fhirclient.models.codeable

## Hinzufügen von Metadaten über die Ressource
Zu den Metadaten gehört u.A. das sogenannte FHIR Profil. In diesem Fall nutzen wir das Herzfrequenz Profil aus der FHIR Spezifikation.

In [4]:
IFrame('https://hl7.org/fhir/heartrate.html#10.1.25.2', width=1200, height=630)

Jede FHIR Ressource hat Metadaten

In [5]:
IFrame('https://www.hl7.org/fhir/resource.html#resource', width=1200, height=330)

In [6]:
from fhirclient.models import meta

In [7]:
obs_meta = meta.Meta()
obs_meta.profile = ['http://hl7.org/fhir/StructureDefinition/heartrate']
my_observation.meta = obs_meta

## Festlegen des Status dieser Observation

In [8]:
status = 'final'
my_observation.status = status

## Festlegen einer Kategorie

In [9]:
IFrame('https://hl7.org/fhir/datatypes.html#codeableconcept', width=1200, height=330)

In [10]:
from fhirclient.models import (
    codeableconcept,
    coding
)

In [11]:
IFrame('http://terminology.hl7.org/CodeSystem/observation-category', width=1200, height=330)

In [12]:
category = codeableconcept.CodeableConcept()
category_coding = coding.Coding()
category_coding.system = 'http://terminology.hl7.org/CodeSystem/observation-category'
category_coding.code = 'vital-signs'
category.coding = [category_coding]
my_observation.category = [category]

In [13]:
print(json.dumps(category.as_json(), indent=4))

{
    "coding": [
        {
            "code": "vital-signs",
            "system": "http://terminology.hl7.org/CodeSystem/observation-category"
        }
    ]
}


## Vergeben eines Codes aus einer Terminologie, der die Bedeutung 'Herzfrequenz' repräsentiert

In [14]:
code = codeableconcept.CodeableConcept()
code_coding = coding.Coding()
code_coding.system = 'http://loinc.org'
code_coding.code = '8867-4'
code_coding.display = 'Heart rate'
code.coding = [code_coding]
my_observation.code = code

In [15]:
print(json.dumps(code.as_json(), indent=4))

{
    "coding": [
        {
            "code": "8867-4",
            "display": "Heart rate",
            "system": "http://loinc.org"
        }
    ]
}


## Festlegen einer Referenz auf die zuvor erstellte Patienten-Ressource

In [16]:
from fhirclient.models import fhirreference
import requests

### GET zuvor erstellte Patienten-Ressource

In [17]:
# fhir_test_server = 'https://server.fire.ly'
fhir_test_server = 'http://hapi.fhir.org/baseR4'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

response = requests.get(url=f'{fhir_test_server}/Patient?family=Bach', headers=headers)

In [18]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

### ID des zuvor erstellten Patientens holen

In [19]:
# Response Text als Python Dictionary holen
response_dict = json.loads(response.text)
response_dict

{'resourceType': 'Bundle',
 'id': 'a0e2808c-57be-4e07-84b6-753130978957',
 'meta': {'lastUpdated': '2022-06-15T14:27:21.432+00:00'},
 'type': 'searchset',
 'total': 9,
 'link': [{'relation': 'self',
   'url': 'http://hapi.fhir.org/baseR4/Patient?family=Bach'}],
 'entry': [{'fullUrl': 'http://hapi.fhir.org/baseR4/Patient/2711466',
   'resource': {'resourceType': 'Patient',
    'id': '2711466',
    'meta': {'versionId': '2',
     'lastUpdated': '2022-03-24T23:47:04.041+00:00',
     'source': '#O2QPQ6QFxFx46R2b'},
    'text': {'status': 'generated',
     'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Johann Sebastian <b>BACH </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>14 December 1964</span></td></tr></tbody></table></div>'},
    'name': [{'use': 'official',
      'family': 'Bach',
      'given': ['Johann', 'Sebastian']}],
    'birthDate': '1964-12-14'},
   'search': {'mode': 'match'}},
  {'fullUrl': 'http://hapi.fhi

In [20]:
response_dict['entry'][0]['resource']['id']

'2711466'

In [21]:
subject = fhirreference.FHIRReference()
pat_id = response_dict['entry'][0]['resource']['id']
subject.reference = f'Patient/{pat_id}'
my_observation.subject = subject

In [22]:
print(json.dumps(subject.as_json(), indent=4))

{
    "reference": "Patient/2711466"
}


## Angeben des Zeitpunktes, wann die Observation gemacht wurde

In [23]:
from fhirclient.models import fhirdate

In [24]:
date = fhirdate.FHIRDate('2020-11-24')
my_observation.effectiveDateTime = date

In [25]:
print(json.dumps(date.as_json(), indent=4))

"2020-11-24"


## Angabe des gemessenen Wertes

In [26]:
from fhirclient.models import quantity

In [27]:
measured_quantity = quantity.Quantity()
measured_quantity.value = 60
measured_quantity.unit = 'beats per minute'
measured_quantity.code = '/min'
measured_quantity.system = 'http://unitsofmeasure.org'
my_observation.valueQuantity = measured_quantity

In [28]:
print(json.dumps(measured_quantity.as_json(), indent=4))

{
    "code": "/min",
    "system": "http://unitsofmeasure.org",
    "unit": "beats per minute",
    "value": 60
}


In [35]:
print(json.dumps(my_observation.as_json(), indent=4))

{
    "meta": {
        "profile": [
            "http://hl7.org/fhir/StructureDefinition/heartrate"
        ]
    },
    "category": [
        {
            "coding": [
                {
                    "code": "vital-signs",
                    "system": "http://terminology.hl7.org/CodeSystem/observation-category"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "code": "8867-4",
                "display": "Heart rate",
                "system": "http://loinc.org"
            }
        ]
    },
    "effectiveDateTime": "2020-11-24",
    "status": "final",
    "subject": {
        "reference": "Patient/2711466"
    },
    "valueQuantity": {
        "code": "/min",
        "system": "http://unitsofmeasure.org",
        "unit": "beats per minute",
        "value": 60
    },
    "resourceType": "Observation"
}


## Die Observation-Ressource validieren und an den FHIR Testserver senden

In [38]:
# fhir_test_server = 'https://server.fire.ly'
fhir_test_server = 'http://hapi.fhir.org/baseR4'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

In [39]:
response = requests.post(f'{fhir_test_server}/Observation/$validate', headers = headers, data = json.dumps(my_observation.as_json()))
data = response.json()

In [40]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [41]:
print(json.dumps(data, indent=4))

{
    "resourceType": "OperationOutcome",
    "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><h1>Operation Outcome</h1><table border=\"0\"><tr><td style=\"font-weight: bold;\">WARNING</td><td>[Observation, Line 1, Col 2]</td><td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>\n\t\t\t</tr>\n\t\t\t<tr>\n\t\t\t\t<td style=\"font-weight: bold;\">WARNING</td>\n\t\t\t\t<td>[Observation.code, Line 1, Col 223]</td>\n\t\t\t\t<td><pre>None of the codings provided are in the value set 'Vital Signs' (http://hl7.org/fhir/ValueSet/observation-vitalsignresult), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8867-4)</pre></td>\n\t\t\t</tr>\n\t\t\t<tr>\n\t\t\t\t<td style=\"font-weight: bold;\">ERROR</td>\n\t\t\t\t<td>[Observation.code.coding

In [42]:
response = requests.post(f'{fhir_test_server}/Observation', headers = headers, data = json.dumps(my_observation.as_json()))
data = response.json()

In [43]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [44]:
print(json.dumps(data, indent=4))

{
    "resourceType": "Observation",
    "id": "3259849",
    "meta": {
        "versionId": "1",
        "lastUpdated": "2022-06-15T14:33:56.102+00:00",
        "profile": [
            "http://hl7.org/fhir/StructureDefinition/heartrate"
        ]
    },
    "status": "final",
    "category": [
        {
            "coding": [
                {
                    "system": "http://terminology.hl7.org/CodeSystem/observation-category",
                    "code": "vital-signs"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "8867-4",
                "display": "Heart rate"
            }
        ]
    },
    "subject": {
        "reference": "Patient/2711466"
    },
    "effectiveDateTime": "2020-11-24",
    "valueQuantity": {
        "value": 60,
        "unit": "beats per minute",
        "system": "http://unitsofmeasure.org",
        "code": "/min"
    }
}
